In [1]:
from utils import *
import numpy as np
import time
from numba import njit, prange

In [2]:
d = 1                       # Dimensionality
s = 3                       # Number of discrete value for each dimension
signal_length = 50000       # signal length
lda = np.log(signal_length) # penalty

In [3]:
signal = np.column_stack([np.random.uniform(0, 2 * np.pi, signal_length) for _ in range(d)])

In [4]:
# Discrete means set
angles = np.linspace(0, 2 * np.pi, s, endpoint=False)
grids = np.meshgrid(*([angles] * d), indexing='ij')
Theta = np.stack(grids, axis=-1).reshape(-1, d)

In [5]:
# @njit(parallel=True)
# def min_axis1(arr):
#     nrows = arr.shape[0]
#     result = np.empty(nrows, dtype=arr.dtype)
#     for i in prange(nrows):
#         row_min = arr[i, 0]
#         for j in range(1, arr.shape[1]):
#             if arr[i, j] < row_min:
#                 row_min = arr[i, j]
#         result[i] = row_min
#     return result

In [6]:
# @njit(parallel=True)
# def compute_diff(cumsum_costs, start, end):
#     n = start.shape[0]
#     n_features = cumsum_costs.shape[1]
#     result = np.empty((n, n_features), dtype=cumsum_costs.dtype)
#     end_val = cumsum_costs[end, :]
#     for i in prange(n):
#         for j in range(n_features):
#             result[i, j] = end_val[j] - cumsum_costs[start[i], j]
#     return result

In [7]:
# def L(start, end, cumsum_costs):
#     times = {}

#     t0 = time.perf_counter()
#     diff = compute_diff(cumsum_costs, start, end)
#     t1 = time.perf_counter()
#     times['subtraction'] = t1 - t0

#     t2 = time.perf_counter()
#     # diff = np.ascontiguousarray(diff, dtype=np.float32)
#     min_vals = min_axis1(diff)
#     t3 = time.perf_counter()
#     times['min'] = t3 - t2

#     return min_vals, times

In [8]:
@njit(parallel=True)
def L(start, end, cumsum_costs):
    n_thetas = cumsum_costs.shape[1]
    L_vals = np.empty(start.shape[0], dtype=cumsum_costs.dtype)
    for i in prange(start.shape[0]):
        row_min = cumsum_costs[end, 0] - cumsum_costs[start[i], 0]
        for j in range(1, n_thetas):
            val = cumsum_costs[end, j] - cumsum_costs[start[i], j]
            if val < row_min:
                row_min = val
        L_vals[i] = row_min
    return L_vals

In [9]:
# def L(start, end, cumsum_costs):
#     t0 = time.perf_counter()
#     min_vals = compute_min_diff(cumsum_costs, start, end)
#     t1 = time.perf_counter()
#     times = t1 - t0
#     return min_vals, times

In [10]:
# def L(start, end, cumsum_costs):
#     return compute_min_diff(cumsum_costs, start, end)

In [11]:
# def L(start, end, cumsum_costs):
#     times = {}

#     t0 = time.perf_counter()
#     diff = cumsum_costs[end] - cumsum_costs[start]
#     t1 = time.perf_counter()
#     times['subtraction'] = t1 - t0

#     t2 = time.perf_counter()
#     diff = np.ascontiguousarray(diff, dtype=np.float32)
#     min_vals = min_axis1(diff)
#     t3 = time.perf_counter()
#     times['min'] = t3 - t2

#     return min_vals, times

In [12]:
# def L(start, end, cumsum_costs):
#     times = {}

#     t0 = time.perf_counter()
#     diff = cumsum_costs[end] - cumsum_costs[start]
#     t1 = time.perf_counter()
#     times['subtraction'] = t1 - t0

#     t2 = time.perf_counter()
#     min_vals = np.min(diff, axis=1)
#     t3 = time.perf_counter()
#     times['min'] = t3 - t2

#     return min_vals, times

In [13]:
# @njit(parallel=True)
# def compute_V(C_sub, lda, t, cumsum_costs):
#     n = np.arange(t).shape[0]
#     n_features = cumsum_costs.shape[1]
#     L_vals = np.empty(n, dtype=cumsum_costs.dtype)
#     end_val = cumsum_costs[t, :]
#     for i in prange(n):
#         row_min = end_val[0] - cumsum_costs[np.arange(t)[i], 0]
#         for j in range(1, n_features):
#             val = end_val[j] - cumsum_costs[np.arange(t)[i], j]
#             if val < row_min:
#                 row_min = val
#         L_vals[i] = row_min

#     V = np.empty(len(C_sub))
#     for i in prange(len(C_sub)):
#         V[i] = C_sub[i] + lda + L_vals[i]
#     return V

In [14]:
@njit(parallel=True)
def compute_V(C_sub, lda, L_vals):
    V = np.empty(len(C_sub))
    for i in prange(len(C_sub)):
        V[i] = C_sub[i] + lda + L_vals[i]
    return V

In [15]:
def opart(signal, Theta, lda):
    # Step 1: Get cumsum_costs
    start = time.perf_counter()
    cumsum_costs = get_cumsum(signal, Theta)
    end = time.perf_counter()
    print(f"[Step 1] get_cumsum: {end - start:.9f} seconds")

    # Step 2: Initialize and compute C and tau_star (Dynamic Programming)
    C = np.zeros(len(signal) + 1)
    C[0] = -lda
    tau_star = np.zeros(len(signal) + 1, dtype=int)

    dp_start = time.perf_counter()
    for t in range(1, len(signal) + 1):
        loop_start = time.perf_counter()
        
        # l_start = time.perf_counter()
        # L_vals, times = L(np.arange(t), t, cumsum_costs)
        # l_end = time.perf_counter()
        
        v_start = time.perf_counter()
        V = compute_V(C[:t], lda, L(np.arange(t), t, cumsum_costs))
        v_end = time.perf_counter()
        
        min_start = time.perf_counter()
        tau_star[t] = np.argmin(V)
        C[t] = V[tau_star[t]]
        min_end = time.perf_counter()

        loop_end = time.perf_counter()

        if t == len(signal):
            print(f"[Step 2 | t={t}] Total: {loop_end - loop_start:.9f} s")
            # print(f"    L total: {l_end - l_start:.9f} s ({times:.9f} s)")
            print(f"    V calc: {v_end - v_start:.9f} s")
            print(f"    min/argmin: {min_end - min_start:.9f} s")

    dp_end = time.perf_counter()
    print(f"[Step 2] Total dynamic programming loop: {dp_end - dp_start:.9f} seconds")

    # Step 3: Trace back changepoints
    start = time.perf_counter()
    chpnts = trace_back(tau_star[1:])
    end = time.perf_counter()
    print(f"[Step 3] trace_back: {end - start:.9f} seconds")

    # Step 4: Compute signal means
    start = time.perf_counter()
    signal_mean = get_signal_mean(Theta, cumsum_costs, chpnts)
    end = time.perf_counter()
    print(f"[Step 4] get_signal_mean: {end - start:.9f} seconds")

In [16]:
opart(signal, Theta, lda)

[Step 1] get_cumsum: 0.003198200 seconds
[Step 2 | t=50000] Total: 0.000164500 s
    V calc: 0.000144200 s
    min/argmin: 0.000019800 s
[Step 2] Total dynamic programming loop: 6.089110300 seconds
[Step 3] trace_back: 0.013099700 seconds
[Step 4] get_signal_mean: 0.031860900 seconds


In [17]:
# use numpy

# [Step 1] get_cumsum: 0.002798000 seconds
# [Step 2 | t=50000] Total: 0.002385200 s
#     L total: 0.002269000 s (subtraction: 0.000880500 s, min: 0.001356300 s)
#     V calc: 0.000085300 s
#     min/argmin: 0.000029900 s
# [Step 2] Total dynamic programming loop: 60.937032100 seconds
# [Step 3] trace_back: 0.010774600 seconds
# [Step 4] get_signal_mean: 0.022541000 seconds

In [18]:
# use numba

# [Step 1] get_cumsum: 0.002440000 seconds
# [Step 2 | t=50000] Total: 0.001770200 s
#     L total: 0.001592200 s (subtraction: 0.001259500 s, min: 0.000296800 s)
#     V calc: 0.000128100 s
#     min/argmin: 0.000048000 s
# [Step 2] Total dynamic programming loop: 48.926414600 seconds
# [Step 3] trace_back: 0.013825300 seconds
# [Step 4] get_signal_mean: 0.032466900 seconds

In [19]:
# use numba for both subtraction and min in L

# [Step 1] get_cumsum: 0.002755500 seconds
# [Step 2 | t=50000] Total: 0.000528600 s
#     L total: 0.000399100 s (subtraction: 0.000048800 s, min: 0.000263100 s)
#     V calc: 0.000094600 s
#     min/argmin: 0.000034100 s
# [Step 2] Total dynamic programming loop: 15.375155100 seconds
# [Step 3] trace_back: 0.013910700 seconds
# [Step 4] get_signal_mean: 0.032910500 seconds

In [20]:
# use numba for both subtraction and min in L and V_cal
# [Step 1] get_cumsum: 0.003164400 seconds
# [Step 2 | t=50000] Total: 0.000407600 s
#     L total: 0.000326400 s (subtraction: 0.000045800 s, min: 0.000247500 s)
#     V calc: 0.000045900 s
#     min/argmin: 0.000034700 s
# [Step 2] Total dynamic programming loop: 14.413672100 seconds
# [Step 3] trace_back: 0.013507100 seconds
# [Step 4] get_signal_mean: 0.031638500 seconds

In [21]:
# use numba for both subtraction and min in L and V_cal and not # diff = np.ascontiguousarray(diff, dtype=np.float32)
# [Step 1] get_cumsum: 0.002512900 seconds
# [Step 2 | t=50000] Total: 0.000144500 s
#     L total: 0.000096400 s (subtraction: 0.000028100 s, min: 0.000027000 s)
#     V calc: 0.000013900 s
#     min/argmin: 0.000033600 s
# [Step 2] Total dynamic programming loop: 7.322234600 seconds
# [Step 3] trace_back: 0.013409200 seconds
# [Step 4] get_signal_mean: 0.032012500 seconds

In [22]:
# using numba combine L
# [Step 1] get_cumsum: 0.002739100 seconds
# [Step 2 | t=50000] Total: 0.000150200 s
#     L total: 0.000104000 s (subtraction: 0.000027400 s, min: 0.000027400 s)
#     V calc: 0.000012900 s
#     min/argmin: 0.000032700 s
# [Step 2] Total dynamic programming loop: 6.323910100 seconds
# [Step 3] trace_back: 0.014082400 seconds
# [Step 4] get_signal_mean: 0.031577500 seconds

In [23]:
# combine min and argmin
# [Step 1] get_cumsum: 0.005495400 seconds
# [Step 2 | t=50000] Total: 0.000147100 s
#     L total: 0.000105300 s (combined: 0.000000200 s)
#     V calc: 0.000021400 s
#     min/argmin: 0.000019500 s
# [Step 2] Total dynamic programming loop: 5.406089200 seconds
# [Step 3] trace_back: 0.013645700 seconds
# [Step 4] get_signal_mean: 0.031484500 seconds